# AT&T SPAM DETECTOR

<img src="img/image.jpg" alt="Image" width="30%" height="30%">

## PARTIE 1 : CHARGEMENT DES DONNEES

In [30]:
# Importation des librairies
import pandas as pd
import tensorflow as tf
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from plotly import graph_objects as go
import plotly.express as px

In [31]:
# Importation des données
data = pd.read_csv('src/spam.csv', encoding = 'latin-1')

# Concaténer les colonnes si 'Unnamed' n'est pas NaN, puis supprimer les colonnes Unnamed
concat_function = lambda row: ' '.join([str(row['v2'])] + [str(row[col]) for col in ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'] if pd.notna(row[col])])

data['sms'] = data.apply(concat_function, axis=1)

data = data.drop(['v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)

display(data)

,v1,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [32]:
# Affichage graphique des proportions
value_counts = data['v1'].value_counts(normalize=True)

px.pie(
    names = value_counts.index, 
    values = value_counts.values, 
    title='Proportion de spam et ham (non spam)',
    width = 600)

Le jeu de données AT&T se compose de 5572 lignes. Avec une proportion de 13.4% de spam, on a un déséquilibre de classe modéré. On verra avec notre modèle de deep learning si le nombre de données est suffisant pour obtenir de bons résultats.

## PARTIE 2 : MODELISATION DEEP LEARNING

## a) Preprocessing

In [33]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='OOV')

tokenizer.fit_on_texts(data['sms'])

data["sms_encoded"] = tokenizer.texts_to_sequences(data['sms'])

data['v1'] = data['v1'].map({'ham': 0, 'spam': 1})

display(data)

,v1,sms,sms_encoded
0,0,"Go until jurong point, crazy.. Available only ...","[50, 473, 4436, 844, 757, 660, 65, 9, 1329, 88..."
1,0,Ok lar... Joking wif u oni...,"[47, 338, 1501, 474, 7, 1942]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,"[48, 491, 9, 20, 5, 800, 903, 3, 178, 1943, 12..."
3,0,U dun say so early hor... U c already then say...,"[7, 249, 151, 24, 384, 3000, 7, 140, 155, 58, ..."
4,0,"Nah I don't think he goes to usf, he lives aro...","[1026, 2, 98, 108, 70, 492, 3, 963, 70, 1946, ..."
...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,"[41, 10, 6, 432, 64, 40, 18, 562, 20, 200, 7, ..."
5568,0,Will Ì_ b going to esplanade fr home?,"[34, 117, 185, 76, 3, 2049, 865, 80]"
5569,0,"Pity, * was in mood for that. So...any other s...","[9010, 61, 9, 1328, 13, 21, 24, 107, 252, 9011]"
5570,0,The guy did some bitching but I acted like i'd...,"[6, 536, 114, 116, 9012, 25, 2, 4375, 56, 904,..."


In [34]:
data_pad = tf.keras.preprocessing.sequence.pad_sequences(data['sms_encoded'], padding="post")

X_train, X_val, y_train, y_val = train_test_split(data_pad, data['v1'], test_size=0.3, random_state=0)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))

train_ds = train_ds.shuffle(len(train_ds)).batch(64)
val_ds = val_ds.shuffle(len(val_ds)).batch(64)

next(iter(train_ds))

(<tf.Tensor: shape=(64, 189), dtype=int32, numpy=
 array([[  39, 1571,  576, ...,    0,    0,    0],
        [ 191,   25,  478, ...,    0,    0,    0],
        [   2, 8749,  713, ...,    0,    0,    0],
        ...,
        [   2,  207,  192, ...,    0,    0,    0],
        [  30,   80,  183, ...,    0,    0,    0],
        [1005,   58,   10, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(64,), dtype=int64, numpy=
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       dtype=int64)>)

## b) Entrainement du modèle

In [35]:
# Définition du modèle
embedding_dim = 32
vocab_size = 10000

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_shape = [data_pad.shape[1],], name="embedding", mask_zero=True), 
  tf.keras.layers.LSTM(units=64, return_sequences=False),
  tf.keras.layers.Dense(1, activation="sigmoid") 
])

# Choix de l'optimizer, de la loss et des métriques
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 189, 32)           320032    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 344,929
Trainable params: 344,929
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Entrainement du modèle
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10)

Epoch 1/10
61/61 [==============================] - 14s 66ms/step - loss: 0.4208 - accuracy: 0.8838 - val_loss: 0.1529 - val_accuracy: 0.9516
Epoch 2/10
61/61 [==============================] - 3s 42ms/step - loss: 0.0839 - accuracy: 0.9800 - val_loss: 0.0699 - val_accuracy: 0.9809
Epoch 3/10
61/61 [==============================] - 3s 43ms/step - loss: 0.0347 - accuracy: 0.9928 - val_loss: 0.0467 - val_accuracy: 0.9862
Epoch 4/10
61/61 [==============================] - 3s 32ms/step - loss: 0.0180 - accuracy: 0.9969 - val_loss: 0.0432 - val_accuracy: 0.9886
Epoch 5/10
61/61 [==============================] - 2s 40ms/step - loss: 0.0096 - accuracy: 0.9987 - val_loss: 0.0424 - val_accuracy: 0.9886
Epoch 6/10
61/61 [==============================] - 2s 40ms/step - loss: 0.0076 - accuracy: 0.9987 - val_loss: 0.0463 - val_accuracy: 0.9856
Epoch 7/10
61/61 [==============================] - 3s 31ms/step - loss: 0.0030 - accuracy: 0.9997 - val_loss: 0.0581 - val_accuracy: 0.9856
Epoch 8/10
6

In [37]:
# # Sauvegarde du modèle (fichier h5)
# model.save('src/model.h5')

# # Sauvegarde de l'historique des epochs
# json.dump(model.history.history, open("src/model_history.json", 'w'))

In [38]:
# Chargement du modèle et de l'historique des epochs
model = tf.keras.models.load_model('src/model.h5')

with open("src/model_history.json", 'r') as file:
    loaded_history = json.load(file)

### c) Résultats

In [39]:
fig = go.Figure(data=[
                      go.Scatter(
                          y=loaded_history["loss"],
                          name="Training loss",
                          mode="lines"),
                      go.Scatter(
                          y=loaded_history["val_loss"],
                          name="Validation loss",
                          mode="lines")
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()

In [40]:
fig = go.Figure(data=[
                      go.Scatter(
                          y=loaded_history["accuracy"],
                          name="Training accuracy",
                          mode="lines"),
                      go.Scatter(
                          y=loaded_history["val_accuracy"],
                          name="Validation accruracy",
                          mode="lines")
])
fig.update_layout(
    title='Training and val accuracy across epochs',
    xaxis_title='epochs',
    yaxis_title='Accuracy'    
)
fig.show()

In [41]:
# Prédictions
predictions_train = (model.predict(X_train) > 0.5).astype(int)
predictions_val = (model.predict(X_val) > 0.5).astype(int)

53/53 [==============================] - 0s 6ms/step


In [42]:
# Matrice de confusion train set
fig = px.imshow(confusion_matrix(y_train, predictions_train),
                labels=dict(x="Prédictions", y="Vraies valeurs"),
                x=['Prediction : ham', 'Prediction : spam'],
                y=['Vrai : ham', 'Vrai : spam'],
                color_continuous_scale='viridis',
                title = 'Matrice de confusion (train set)',
                text_auto = True,
                width = 600)
fig.show()

# Matrice de confusion validation set
fig = px.imshow(confusion_matrix(y_val, predictions_val),
                labels=dict(x="Prédictions", y="Vraies valeurs"),
                x=['Prediction : ham', 'Prediction : spam'],
                y=['Vrai : ham', 'Vrai : spam'],
                color_continuous_scale='viridis',
                title = 'Matrice de confusion (val set)',
                text_auto=True,
                width = 600)
fig.show()

In [46]:
# Rapport de classification sur le jeu de test
print("Rapport de classification :\n", classification_report(y_val, predictions_val, digits = 3))

Rapport de classification :
               precision    recall  f1-score   support

           0      0.990     0.999     0.994      1434
           1      0.996     0.937     0.965       238

    accuracy                          0.990      1672
   macro avg      0.993     0.968     0.980      1672
weighted avg      0.990     0.990     0.990      1672



J'ai fait le choix de partir sur un modèle simple avec 1 couche embedding et 1 seule couche LSTM. L'objectif étant de tester différentes formules si le résultat n'est pas suffisant.

Au global le modèle est très performant. On a une précision de 99%, le déséquilibre de classe n'est pas un soucis. Seulement 16 erreurs ont été constatés. En consultant la matrice de confusion, on remarque que les erreurs sont principalement sur les faux négatifs.

On pourrait demander à AT&T quel est le taux de précision qu'ils souhaitent obtenir. De mon point de vue, on est déjà sur un modèle fiable et utilisable.

### d) Améliorations

In [44]:
# Création d'un dataframe avec les valeurs y_val et les valeurs prédites
df_fpfn = pd.concat([y_val.reset_index(), pd.Series(predictions_val.ravel())], axis = 1)
df_fpfn = df_fpfn.set_index('index').rename_axis(index=None)
df_fpfn.columns = ['y_val', 'Predictions']

# Fonction pour repérer les faux positifs et négatifs
def check_false_predictions(row):
    if row['Predictions'] == 1 and row['y_val'] == 0:
        return 'Faux Positif'
    elif row['Predictions'] == 0 and row['y_val'] == 1:
        return 'Faux Negatif'

# Application de la fonction + filtres
df_fpfn['Predictions_result'] = df_fpfn.apply(check_false_predictions, axis = 1)
df_fpfn = df_fpfn[df_fpfn['Predictions_result'].notnull()]
df_fpfn = df_fpfn.loc[:,'Predictions_result']

In [45]:
# Création du dataframe avec les SMS des faux positifs et négatifs
df_final = pd.concat([data, df_fpfn], axis = 1)
df_final = df_final[df_final['Predictions_result'].notnull()]
df_final = df_final.loc[:,['sms', 'Predictions_result']]
df_final = df_final.sort_values('Predictions_result')
df_final = df_final.reset_index(drop = True)

display(df_final)

,sms,Predictions_result
0,Hi I'm sue. I am 20 years old and work as a la...,Faux Negatif
1,"SMS. ac JSco: Energy is high, but u may not kn...",Faux Negatif
2,CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...,Faux Negatif
3,Reply to win å£100 weekly! Where will the 2006...,Faux Negatif
4,Back 2 work 2morro half term over! Can U C me ...,Faux Negatif
5,Babe: U want me dont u baby! Im nasty and have...,Faux Negatif
6,Sorry! U can not unsubscribe yet. THE MOB offe...,Faux Negatif
7,LookAtMe!: Thanks for your purchase of a video...,Faux Negatif
8,Sorry I missed your call let's talk when you h...,Faux Negatif
9,ringtoneking 84484,Faux Negatif


Notre modèle pourrait bien entendu être amélioré. Une entreprise comme AT&T cherche probablement à se rapprocher au maximum d'un modèle parfait pour satisfaire ses utilisateurs.

Premièrement, j'ai extrait tous les faux postitifs et faux négatifs que donnent notre modèle. On pourrait les analyser et trouver des raisons à cette mauvaise prédiction :
- Pour les faux négatifs, dans l'ensemble peut voir que les phrases semblent assez normal et écrites sans fautes.
- Pour la ligne en faux positif, la répétition de lettres 'Ujhhhhhhhh' est probablement la cause de la prédiction spam.

Secondement, le transfert learning à partir de modèles entrainés sur des jeux de données plus conséquent pourrait donner un résultat plus satisfaisant, et s'adaptant à de nouveaux types de spam.